# Modul #4 üçün Tapşırıqlar: 2-ci Hissə

In [ ]:
!pip install scikit-optimize

In [18]:
# Lazım olan kitabxanalar
from sklearn.preprocessing import RobustScaler, MinMaxScaler, MaxAbsScaler, StandardScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score, train_test_split, RepeatedStratifiedKFold, RepeatedKFold
from sklearn.datasets import make_regression, make_classification
from sklearn.feature_selection import SelectFromModel, SelectFdr
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import warnings
import sklearn
import skopt

In [5]:
pd.set_option('display.max_columns', None)
warnings.filterwarnings(action = 'ignore')
sklearn.set_config(display = 'diagram')
np.random.seed(seed = 42)

In [6]:
def generate_data(regression = True, n_observations = None, n_features = None, n_missing = None):
    if regression:
        X, Y = make_regression(n_samples = n_observations, n_features = n_features, random_state = 42)
    else:
        X, Y = make_classification(n_samples = n_observations, n_features = n_features, random_state = 42)

    column_names = [f'feature_{x}' for x in range(1, n_features + 1)]

    X = pd.DataFrame(data = X)
    Y = pd.Series(data = Y, name = 'target').to_frame()

    X.columns = column_names

    rock = ('Rock ' * 25).split()
    jazz = ('Jazz ' * 25).split()
    metal = ('Metal ' * 25).split()
    lyric = ('Lyric ' * 25).split()

    X[f'feature_{n_features + 1}'] = rock + jazz + metal + lyric

    data_frame = pd.concat(objs = [X, Y], axis = 1)

    assert data_frame.shape[0] == n_observations
    assert data_frame.shape[1] == n_features + 2

    column_names = column_names + [column for column in X.columns.tolist() if column not in column_names]

    for feature in column_names:
        data_frame.loc[data_frame.index.isin(values = data_frame[feature].sample(n = n_missing).index), feature] = np.nan

    data_frame = data_frame.sample(frac = 1.0, random_state = 42, ignore_index = True)

    return data_frame

In [7]:
# Sinifləndirmə datasetin yaradılması
df = generate_data(regression = False, n_observations = 100, n_features = 10, n_missing = 50)

# İlk beş sətrin göstərilməsi
df.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,target
0,NaN,NaN,NaN,NaN,-0.219101,NaN,NaN,NaN,NaN,0.235615,Lyric,0
1,NaN,NaN,NaN,0.703852,NaN,-0.727137,0.620672,-1.335344,NaN,0.177701,NaN,1
2,NaN,2.304500,1.83991,1.339702,NaN,NaN,0.444263,1.159330,NaN,-0.360966,Metal,1
3,NaN,NaN,NaN,0.211646,-0.478749,0.222134,NaN,NaN,1.255756,NaN,NaN,1
4,NaN,-0.713525,NaN,NaN,NaN,NaN,-0.525755,0.150394,-2.123896,-0.759133,NaN,1


In [8]:
# Asılı olmayan dəyişənlərin filterlənməsi
X = df.drop(columns = 'target')

# Asılı olan dəyişənin filterlənməsi
Y = df.target

# Asılı olmayan dəyişənlərin və asılı olan dəyişənin train & test setə bölünməsi
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

***
### Tapşırıq #1:
- LogisticRegression() alqoritmasından istifadə edərək model dəyişəni yarat (solver = 'liblinear', max_iter = 500 parametrlərindən istifadə et)
- Kateqorik dəyişənlər üçün boru yarat və borunun ilk mərhələsində boş xanaları ən çöx təkrarlanan dəyərlə əvəz et, ikinci mərhələsində isə kateqorik dəyişənin unikal dəyərlərini numerik dəyişənlərə çevir (handle_unknown = 'ignore' istifadə et)
- Numerik dəyişənlər üçün boru yarat və borunun ilk mərhələsində boş xanaları median dəyərlə əvəz et, ikinci mərhələsində isə numerik dəyişənlərin dəyərlərini eyni şkalaya sal
- Kateqorik və numerik boruları birləşdir dəyişənlər üçün əməliyyatlar toplusu yarat
- Sinifləndirmə borusu yarat, borunun ilk mərhələsində əməliyyatlar toplusu, ikinci mərhələsində SelecFromModel() metodu, son mərhələsində isə algorithm dəyişəni olsun

In [20]:
# LogisticRegression modelinin qurulması
algorithm = LogisticRegression(solver = 'liblinear', max_iter = 500, random_state = 1)

# Kateqorik dəyişənlər üçün borunun yaradılması
categoric_pipeline = Pipeline(steps = [('imputer', SimpleImputer(strategy = 'most_frequent')), ('ohe', OneHotEncoder(handle_unknown = 'ignore'))])

# Numerik dəyişənlər üçün borunun yaradılması
numeric_pipeline = Pipeline(steps = [('imputer', SimpleImputer(strategy = 'median')), ('scaler', RobustScaler())])

# Kateqorik və numerik borunun birləşdirilməsi
feature_transformer = ColumnTransformer(transformers = [
    ('categoric_transformer', categoric_pipeline, X_train.select_dtypes(include = 'object').columns.tolist()),
    ('numeric_transformer', numeric_pipeline, X_train.select_dtypes(include = 'number').columns.tolist())
    ],
    n_jobs = -1)

# Sinifləndirmə borusunun yaradılması
pipe = Pipeline(steps = [('feature_transformer', feature_transformer), ('feature_selector', SelectFromModel(estimator = algorithm)), ('classifier', algorithm)])

# Boru arxitekturasının göstərilməsi
pipe

Pipeline(steps=[('feature_transformer',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('categoric_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['feature_11']),
                                                 ('numeric_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['feature_1', 'feature_2',
                                                   'feature_3', 'feature_4',
                                                   'feature_5', 'feature_6',
                                                   'feature_7', 'feature_8',
                                                   'feature_9',
                                                   'feature_10'])])),
                ('feature_selector',
                 SelectFromModel(estimator=LogisticRegression(max_iter=500,
                                                              random_state=1,
                                                              solver='liblinear'))),
                ('classifier',
                 LogisticRegression(max_iter=500, random_state=1,
                                    solver='liblinear'))])

***
### Tapşırıq #2:
- **apply_bayesian_optimization** funksiyası yarat və funksiyaya 6 parametr ver.
1. model - sinifləndirmə borusu
2. hyperparameters - lügət data strukturunda olan hiper parametrlər
3. n_iterations - Ən optimal hiper parametrlərin tapılmasında istifadə olunan döngü sayı
4. metric - Ən optimal hiper parametrlərin tapılmasında istifadə olunan qiymətləndirmə meyarı
5. train_features - Asılı olmayan train dəyişənləri
6. train_labels - Asılı olan train dəyişəni

> BayesSearchCV alqorithmasını qur və ən optimal hiper parametrlərdən ibarət borunu geri qaytar

In [14]:
from skopt import BayesSearchCV

def apply_bayesian_optimization(model, hyperparameters, n_iterations, metric, train_features, train_labels):
  bayes_search = BayesSearchCV(estimator = model, search_spaces = hyperparameters, n_iter = n_iterations, scoring = metric, n_jobs = -1, cv = RepeatedKFold(random_state = 1), random_state = 1)
  bayes_search.fit(train_features, train_labels)

  pipe = bayes_search.best_estimator_
  return pipe

***
### Tapşırıq #3:
- **search_spaces** adında boş lügət data strukturu yarat
- search_spaces-ə numerik boruda istifadə oluna biləcək şkala alqoritmalarını daxil et
- search_spaces-ə numerik boruda istifadə oluna biləcək əvəzləmə strategyalarını daxil et
- search_spaces-ə kateqorik boruda istifadə oluna biləcək əvəzləmə strategyalarını daxil et
- search_spaces-ə sinifləndirmə borusunda istifadə oluna biləcək ən vacib dəyişənlərin seçilməsi alqoritmlərini daxil et
- search_spaces-ə sinifləndirmə borusunda Logistic Regression alqoritmasının C hiper parametrinin ala biləcəyi dəyərləri daxil et (low = 1e-6, high = 100 parametrlərindən istifadə et)
- search_spaces-ə sinifləndirmə borusunda Logistic Regression alqoritmasının tol hiper parametrinin ala biləcəyi dəyərləri daxil et (low = 1e-6, high = 100 parametrlərindən istifadə et)

In [21]:
# Ən optimal hiper parametrlərin tapılması üçün lügət data struktunun yaradılması
search_spaces = {}
# Hiper parametrlərin lügət data strukturuna daxil edilməsi
search_spaces['feature_transformer__numeric_transformer__scaler'] = skopt.space.Categorical(categories = [RobustScaler(), MinMaxScaler(), MaxAbsScaler(), StandardScaler()])
search_spaces['feature_transformer__numeric_transformer__imputer__strategy'] = skopt.space.Categorical(categories = ['mean', 'median', 'constant', 'most_frequent'])
search_spaces['feature_transformer__categoric_transformer__imputer__strategy'] = skopt.space.Categorical(categories = ['constant', 'most_frequent'])
search_spaces['feature_selector'] = skopt.space.Categorical(categories = [SelectFromModel(estimator = algorithm), SelectFdr()])
search_spaces['classifier__C'] = skopt.space.Real(low = 1e-6, high = 100)
search_spaces['classifier__tol'] = skopt.space.Real(low = 1e-6, high = 100)

# Sinifləndirmə borusunda ən optimal hiper parametrlərin tapılması
best_pipe = apply_bayesian_optimization(model = pipe, hyperparameters = search_spaces, n_iterations = 100, metric = 'accuracy', train_features = X_train, train_labels = y_train)

# Boru arxitekturasının göstərilməsi
best_pipe

Pipeline(steps=[('feature_transformer',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('categoric_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['feature_11']),
                                                 ('numeric_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['feature_1', 'feature_2',
                                                   'feature_3', 'feature_4',
                                                   'feature_5', 'feature_6',
                                                   'feature_7', 'feature_8',
                                                   'feature_9',
                                                   'feature_10'])])),
                ('feature_selector',
                 SelectFromModel(estimator=LogisticRegression(max_iter=500,
                                                              random_state=1,
                                                              solver='liblinear'))),
                ('classifier',
                 LogisticRegression(C=69.24720095900523, max_iter=500,
                                    random_state=1, solver='liblinear',
                                    tol=1e-06))])